# 0.1. Импортируем зависимости

In [63]:
import itertools
import pandas as pd
import psycopg2
from psycopg2.extensions import quote_ident
import re

# 0.2. Подключимся к базе
Предполагается, что соответствующая база и пользователь с соответствующими правами уже созданы, таблиц с наименованиями, конфликтующих с наименованиями таблиц задания в базе нет, подключение таким образом возможно и т.п.

In [64]:
DATABASE = 'db_hw_2'
USER = 'db_hw_2_user'

connection = psycopg2.connect(dbname=DATABASE, user=USER)
connection.autocommit = True
cursor = connection.cursor()

# 1.1. Создадим таблицы с соответствующими структурами

In [65]:
cursor.execute("""
    CREATE TABLE customer_20240101 (
        customer_id int4,
        first_name varchar(50),
        last_name varchar(50),
        gender varchar(30),
        dob varchar(50),
        job_title varchar(50),
        job_industry_category varchar(50),
        wealth_segment varchar(50),
        deceased_indicator varchar(50),
        owns_car varchar(30),
        address varchar(50),
        postcode varchar(30),
        state varchar(30),
        country varchar(30),
        property_valuation int4
    )
""")

cursor.execute("""
    CREATE TABLE transaction_20240101 (
        transaction_id int4,
        product_id int4,
        customer_id int4,
        transaction_date varchar(30),
        online_order varchar(30),
        order_status varchar(30),
        brand varchar(30),
        product_line varchar(30),
        product_class varchar(30),
        product_size varchar(30),
        list_price float4,
        standard_cost float4
    )
""")

# 1.2. Загрузим данные в базу из csv-файлов 
Для этого, после чтения данных из каждого файла, будем формировать и выполнять SQL-запрос `INSERT INTO ...` вида `INSERT INTO table_name ( column_name [, ...] ) VALUES (expression, ...), ...` согласно [официальной документации PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html).

Заодно выведем наименования колонок таблиц.

In [66]:
for filename in ['customer.csv', 'transaction.csv']:
    # прочитаем данные из соответствующего файла
    # NULL-ами (в терминах SQL) будем считать пустые строки и строку "null" в любом регистре
    # типы используем такие, которые позволят избежать нежелательного изменения данных
    # с учётом типов, заданных выше для соответствующих колонок наших таблиц в базе
    df = pd.read_csv(filename, nrows=1, sep=';')
    dtypes = {k: str for k in df.columns}
    dtypes.update(list_price=float, standard_cost=float)
    df = pd.read_csv(
        filename, 
        sep=';',
        na_values=('', ),
        keep_default_na=False,
        decimal=',',
        dtype=dtypes,
    )
    table = f'{filename[:-4]}_20240101'
    df = df.map(lambda x: None if isinstance(x, str) and re.match('^null$', x, flags=re.I) else x)
    df = df.where(pd.notnull(df), None)
    
    # значения для вставки и наименования колонок
    values = list(itertools.chain.from_iterable(df.values.tolist()))
    cols = [i.lower() for i in df.columns]
    del df
    
    # Сформируем (и выполним) запрос INSERT INTO следующего вида
    # INSERT INTO table_name ( column_name [, ...] ) VALUES (expression, ...), ...
    # согласно официальной документации https://www.postgresql.org/docs/current/sql-insert.html
    cols_sql = ', '.join(quote_ident(i, cursor) for i in cols)
    print(f'Колонки таблицы "{table}": {cols_sql}')
    values_sql = ','.join([f'(' + ','.join(['%s'] * len(cols)) + ')'] * (len(values) // len(cols)))
    table_ident = quote_ident(table, cursor)
    cursor.execute(f'INSERT INTO {table_ident} ({cols_sql}) VALUES {values_sql}', values)

Колонки таблицы "customer_20240101": "customer_id", "first_name", "last_name", "gender", "dob", "job_title", "job_industry_category", "wealth_segment", "deceased_indicator", "owns_car", "address", "postcode", "state", "country", "property_valuation"
Колонки таблицы "transaction_20240101": "transaction_id", "product_id", "customer_id", "transaction_date", "online_order", "order_status", "brand", "product_line", "product_class", "product_size", "list_price", "standard_cost"


## Получим полезную информацию о данных для использования в дальнейшем
По наименованиям колонок и в результате осмотра данных в файлах / таблицах видим, что потенциальные ключи тут скорее всего `customer_id` для таблицы `customer_20240101` и `transaction_id` для таблицы `transaction_20240101`, а объединять таблицы скорее всего разумно по колонке `customer_id`, получим дополнительные данные, чтобы подкрепить данные предположения.

In [67]:
cursor.execute("""
    SELECT count(DISTINCT customer_id) = count(*), bool_or(customer_id IS NULL)
    FROM customer_20240101
""")
print(('В таблице customer_20240101 значения customer_id уникальны: %s,'
       + ' есть ли пропуски (null) в customer_id: %s') % cursor.fetchall()[0])
cursor.execute("""
    SELECT count(DISTINCT transaction_id) = count(*), bool_or(transaction_id IS NULL)
    FROM transaction_20240101
""")
print(('В таблице transaction_20240101 значения transaction_id уникальны: %s,'
       ' есть ли пропуски (null) в transaction_id: %s') % cursor.fetchall()[0])
cursor.execute("""
    SELECT count(*), sum((c.customer_id IS NULL)::int)
    FROM transaction_20240101 t LEFT JOIN customer_20240101 c USING (customer_id)
""")
print(('В таблице transaction_20240101 количество записей (строк): %s, записей,'
       ' по которым нет соответствующей записи в customer_20240101,'
       ' если присоединять по равенству customer_id: %s') % cursor.fetchall()[0])

В таблице customer_20240101 значения customer_id уникальны: True, есть ли пропуски (null) в customer_id: False
В таблице transaction_20240101 значения transaction_id уникальны: True, есть ли пропуски (null) в transaction_id: False
В таблице transaction_20240101 количество записей (строк): 20000, записей, по которым нет соответствующей записи в customer_20240101, если присоединять по равенству customer_id: 3


Из вышеполученных результатов видим, что атрибуты `customer_id` и `transaction_id` являются потенциальными ключами таблиц `customer_20240101` и `transaction_20240101` соответственно.
Для надёжности, ниже создадим соответствующие ограничения "первичный ключ", что не получится, если вывод неверен.
Кроме того, видно, что почти всем значениям `customer_id` в таблице `transaction_20240101` соответствует значение `customer_id` в таблице `customer_20240101`, поэтому будем соединять их по равенству значений этой колонки, когда для запроса нам будут нужны данные из обеих таблиц.

In [68]:
cursor.execute('ALTER TABLE customer_20240101 ADD PRIMARY KEY (customer_id)')
cursor.execute('ALTER TABLE transaction_20240101 ADD PRIMARY KEY (transaction_id)')

Получилось наложить соответствующие ограничения "первичный ключ", чем дополнительно подтвердилось, что можем полагаться на то, что ключам customer_id, transaction_id соответствуют уникальные строки в соответствующих таблицах, где они являются первичными ключами, пропусков (null в терминах SQL) в этих полях нет.

Поскольку ниже нам понадобится знать, является ли заказ онлайн-заказом или нет, посмотрим, какие значения бывают в соответствующей колонке.

In [69]:
cursor.execute('SELECT DISTINCT online_order FROM transaction_20240101')
cursor.fetchall()

[(None,), ('True',), ('False',)]

Видим, что для получения онлайн-заказов разумно будет использовать условие `online_order = 'True'`.

Далее посмотрим, какие значения бывают у статуса подтверждённости транзакции.

In [70]:
cursor.execute('SELECT DISTINCT order_status FROM transaction_20240101')
cursor.fetchall()

[('Approved',), ('Cancelled',)]

Видим, что подтверждённые транзакции разумно будет выбирать по значению `order_status = 'Approved'`

# 2. Выполним запросы для вывода разной информации

Результат каждого запроса будем выводить в виде списка кортежей, который возвращает метод `cursor.fetchall` соответствующего объекта из библиотеки `psycopg2`

Ниже в запросах будем использовать оператор `DISTINCT` для вывода только уникальных строк, функцию `to_date` для приведения строк к типу `date`.

## 2.1. Выведем все уникальные бренды, у которых стандартная стоимость выше 1500 долларов

In [71]:
cursor.execute("""
    SELECT DISTINCT brand
    FROM transaction_20240101
    WHERE standard_cost > 1500
        -- кажется нелогичным считать пропуски (NULL) брендами, поэтому их выводить не будем
        AND brand IS NOT NULL
""")
cursor.fetchall()

[('Solex',), ('Giant Bicycles',), ('OHM Cycles',), ('Trek Bicycles',)]

## 2.2. Выведем все подтвержденные транзакции за период с '2017-04-01' по '2017-04-09' включительно

In [80]:
cursor.execute("""
    SELECT transaction_id, transaction_date
    FROM transaction_20240101
    WHERE order_status = 'Approved'
        AND to_date(transaction_date, 'DD.MM.YYYY') BETWEEN '2017-04-01' AND '2017-04-09'
""")
cursor.fetchall()

[(17, '03.04.2017'),
 (19, '06.04.2017'),
 (23, '08.04.2017'),
 (83, '01.04.2017'),
 (89, '04.04.2017'),
 (126, '01.04.2017'),
 (146, '06.04.2017'),
 (154, '06.04.2017'),
 (220, '09.04.2017'),
 (228, '04.04.2017'),
 (280, '08.04.2017'),
 (290, '05.04.2017'),
 (304, '07.04.2017'),
 (526, '02.04.2017'),
 (534, '08.04.2017'),
 (563, '05.04.2017'),
 (669, '07.04.2017'),
 (671, '09.04.2017'),
 (767, '06.04.2017'),
 (898, '03.04.2017'),
 (904, '02.04.2017'),
 (943, '08.04.2017'),
 (948, '09.04.2017'),
 (1044, '01.04.2017'),
 (1049, '07.04.2017'),
 (1073, '05.04.2017'),
 (1074, '05.04.2017'),
 (1125, '07.04.2017'),
 (1161, '02.04.2017'),
 (1187, '06.04.2017'),
 (1197, '04.04.2017'),
 (1235, '04.04.2017'),
 (1242, '03.04.2017'),
 (1254, '04.04.2017'),
 (1275, '08.04.2017'),
 (1294, '09.04.2017'),
 (1342, '03.04.2017'),
 (1418, '08.04.2017'),
 (1497, '08.04.2017'),
 (1503, '07.04.2017'),
 (1593, '06.04.2017'),
 (1639, '08.04.2017'),
 (1653, '02.04.2017'),
 (1686, '03.04.2017'),
 (1695, '03.04.2

## 2.3. Выведем все профессии у клиентов из сферы IT или Financial Services, которые начинаются с фразы 'Senior'
Для такой фильтрации используем оператор `SIMILAR TO` для фильтрации с помощью регулярного выражения в определении SQL стандарта, а такое выражение, соответственно, пропишем в запросе в виде `'Senior\M%'`, чтобы фильтровались наименования, начинающиеся с фразы 'Senior'.

In [73]:
cursor.execute("""
    SELECT DISTINCT job_title
    FROM customer_20240101
    WHERE job_industry_category IN ('IT', 'Financial Services')
        AND job_title SIMILAR TO 'Senior\M%'
""")
cursor.fetchall()

[('Senior Cost Accountant',),
 ('Senior Developer',),
 ('Senior Editor',),
 ('Senior Financial Analyst',),
 ('Senior Quality Engineer',),
 ('Senior Sales Associate',)]

## 2.4. Выведем все бренды, которые закупают клиенты, работающие в сфере Financial Services
Здесь нам понадобятся данные из двух таблиц. Для присоединения используем, например, оператор `USING`, т.к. присоединять мы хотим по условию равенства значений `customer_id` из обеих таблиц (осмотрев данные в файле и по наименованиям колонок таблиц, которые мы выводили выше, несложно сделать вывод, что присоединять нужно по `customer_id`), что данный оператор нам и обеспечит.

Используем `INNER JOIN`, т.к. нас интересуют только случаи наличия соответствующих данных в обеих таблицах.

In [74]:
cursor.execute("""
    SELECT DISTINCT t.brand
    FROM transaction_20240101 t
    INNER JOIN customer_20240101 c
        USING (customer_id)
    WHERE c.job_industry_category = 'Financial Services'
        AND t.brand IS NOT NULL
""")
cursor.fetchall()

[('Giant Bicycles',),
 ('Norco Bicycles',),
 ('OHM Cycles',),
 ('Solex',),
 ('Trek Bicycles',),
 ('WeareA2B',)]

## 2.5. Выведем 10 клиентов, которые оформили онлайн-заказ продукции из брендов 'Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles'
Для разнообразия (и чтобы показать, что мы и так умеем) присоединим по условию без оператора `USING`.

In [75]:
cursor.execute("""
    SELECT DISTINCT c.customer_id, c.first_name, c.last_name
    FROM customer_20240101 c
    INNER JOIN transaction_20240101 t
        ON t.customer_id = c.customer_id
    WHERE t.online_order = 'True'
        AND t.brand IN ('Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles')
    LIMIT 10
""")
cursor.fetchall()

[(1, 'Laraine', 'Medendorp'),
 (2, 'Eli', 'Bockman'),
 (3, 'Arlin', 'Dearle'),
 (5, 'Sheila-kathryn', 'Calton'),
 (6, 'Curr', 'Duckhouse'),
 (7, 'Fina', 'Merali'),
 (8, 'Rod', 'Inder'),
 (11, 'Uriah', 'Bisatt'),
 (12, 'Sawyere', 'Flattman'),
 (13, 'Gabriele', 'Norcross')]

## 2.6. Выведем всех клиентов, у которых нет транзакций
Тут используем `LEFT JOIN`, чтобы в выборке сохранились строки из `customer_20240101`, не соответствующие никаким строкам из `transaction_20240101` по соответствующему условию. Эти строки нас и будут интересовать - это будут строки, относящиеся к клиентам, у которых нет транзакций, что нас и интересует. Отфильтруем их с помощью соответствующего условия, как показано в запросе.

In [79]:
cursor.execute("""
    SELECT c.customer_id, c.first_name, c.last_name
    FROM customer_20240101 c
    LEFT JOIN transaction_20240101 t
        USING (customer_id)
    WHERE t.customer_id IS NULL
""")
cursor.fetchall()

[(852, 'Andie', 'Bonney'),
 (869, 'Addia', 'Abels'),
 (1373, 'Shaylynn', 'Epsley'),
 (2074, 'Roslyn', 'Rawdall'),
 (2660, 'Hunt', 'Scollard'),
 (3229, 'Carrol', 'Gheorghie'),
 (3236, 'Edsel', 'Arguile'),
 (3501, 'Kayla', 'Alentyev'),
 (3502, 'Wilfrid', 'Cleaves'),
 (3503, 'Tootsie', 'Rafter'),
 (3504, 'Keriann', 'Perdue'),
 (3505, 'Dolly', 'Scallan'),
 (3506, 'Mercy', 'Donisi'),
 (3507, 'Christoph', 'Greatreax'),
 (3508, 'Malvin', 'Marre'),
 (3509, 'Janine', 'Hesey'),
 (3510, 'Jemima', 'Izaac'),
 (3511, 'Willem', 'Spraging'),
 (3512, 'Vladimir', 'Abramowsky'),
 (3513, 'Enriqueta', 'Waterhowse'),
 (3514, 'Darcy', 'Ilett'),
 (3515, 'Leelah', 'Pynner'),
 (3516, 'Diarmid', 'Vearnals'),
 (3517, 'Domeniga', 'Terney'),
 (3518, 'Magdaia', 'Beardwell'),
 (3519, 'Aldus', 'Kenningley'),
 (3520, 'Travis', 'Kaszper'),
 (3521, 'Oneida', 'Olivia'),
 (3522, 'Lorri', 'Byrth'),
 (3523, 'Lin', 'Tarver'),
 (3524, 'Waiter', 'Piscopello'),
 (3525, 'Town', 'Yitzhakov'),
 (3526, 'Pail', 'Iacopini'),
 (3527, '

## 2.7. Выведем всех клиентов из IT, у которых транзакции с максимальной стандартной стоимостью
Тут используем подзапрос для получения максимальной стандартной стоимости транзакций, которую будем использовать в соответствующем условии.

In [77]:
cursor.execute("""
    SELECT DISTINCT c.customer_id, c.first_name, c.last_name
    FROM customer_20240101 c
    INNER JOIN transaction_20240101 t
        USING (customer_id)
    WHERE c.job_industry_category = 'IT'
        AND t.standard_cost = (SELECT max(standard_cost) FROM transaction_20240101)
""")
cursor.fetchall()

[(34, 'Jephthah', 'Bachmann'),
 (168, 'Reggie', 'Broggetti'),
 (255, 'Keeley', 'Kruger'),
 (448, 'Robin', 'Crack'),
 (968, 'Wayne', 'Woodfin'),
 (975, 'Goldarina', 'Rzehorz'),
 (1182, 'Carolann', 'Ottiwill'),
 (1210, 'Shandie', 'Sprigg'),
 (1418, 'Valaria', 'Hugh'),
 (1672, 'Sharla', 'Creebo'),
 (1773, 'Nickolas', 'Guittet'),
 (1972, 'Obed', 'Pauler'),
 (2205, 'Birk', 'Elphey'),
 (2341, 'Caterina', 'Scedall'),
 (2638, 'Moritz', 'Brognot'),
 (3086, 'Pieter', 'Gadesby')]

## 2.8. Выведем всех клиентов из сферы IT и Health, у которых есть подтвержденные транзакции за период с '2017-07-07' по '2017-07-17'

In [78]:
cursor.execute("""
    SELECT DISTINCT c.customer_id, c.first_name, c.last_name
    FROM customer_20240101 c
    INNER JOIN transaction_20240101 t
        USING (customer_id)
    WHERE c.job_industry_category IN ('IT', 'Health')
        AND t.order_status = 'Approved'
        AND to_date(transaction_date, 'DD.MM.YYYY') BETWEEN '2017-07-07' AND '2017-07-17'
""")
cursor.fetchall()

[(22, 'Deeanne', 'Durtnell'),
 (28, 'Fee', 'Zellmer'),
 (41, 'Basilius', 'Coupe'),
 (47, 'Matthew', 'Jeaycock'),
 (104, 'Odille', 'Panketh'),
 (235, 'Leona', 'Phateplace'),
 (239, 'Wells', 'Pressman'),
 (249, "D'arcy", 'Slay'),
 (289, 'Modestia', 'Lithgow'),
 (290, 'Giorgio', 'Kevane'),
 (300, 'Malachi', 'Hadcroft'),
 (320, 'Aldous', 'Cubin'),
 (331, 'Ford', 'Groucock'),
 (338, 'Hilliary', 'Littrell'),
 (350, 'Carmela', 'Jesper'),
 (382, 'Issiah', 'Fardell'),
 (414, 'Thorin', 'Grimoldby'),
 (424, 'Dennie', 'Eunson'),
 (428, 'Freida', 'Everil'),
 (503, 'Burr', 'Gerb'),
 (526, 'Ardelle', None),
 (579, 'Cathlene', 'Bellas'),
 (590, 'Ddene', 'Burleton'),
 (612, 'Grange', 'Skillington'),
 (661, 'Cinderella', 'Haslen'),
 (679, 'Yolande', 'Whild'),
 (682, 'Ole', 'Euplate'),
 (685, 'Booth', 'Birkin'),
 (694, 'Lilith', 'Lanning'),
 (708, 'Idalina', 'Wilcot'),
 (712, 'Norine', 'Antonik'),
 (722, 'Stewart', 'Brosoli'),
 (753, 'Josy', 'St. Quentin'),
 (759, 'Mitchel', 'Schimek'),
 (769, 'Cameron',